In [2]:
# -*- coding: utf-8 -*-
import unicodecsv
import os
import pyLDAvis
import pyLDAvis.gensim



from nltk import RegexpTokenizer
from lib.utils import *
from itertools import chain

/usr/local/lib/python2.7/site-packages/numpy/lib/utils.py:99: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  warnings.warn(depdoc, DeprecationWarning)


In [4]:
class data:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
args = data()
args.filename = "soya_month.csv"
args.stemmer = "lemma"
args.dictionary = "english"
args.num_iter = 1000
args.num_top_words = 8
args.num_topics = 10
args.model = "lda"
args.override = False
args.input_field = "contents"


dir = os.getcwd()
model_dir = os.path.join(dir, 'models/')
dataset_dir = os.path.join(dir, 'datasets/')
dictionary_dir = os.path.join(dir, 'dictionaries/')
executable_dir = os.path.join(dir, 'executables/')
my_timeslices = [500,500,500,500,500, 346]

model_filename = os.path.join(model_dir, get_model_with_arguments_filename(args))
is_english_word = load_from_dictionary(args.dictionary)

print model_filename

/Users/Hii/Projects/news_scraper/models/soya_month_lemma_1000_8_10_lda_english


In [5]:
dataset_filepath = os.path.join(dataset_dir, args.filename)
f = open(dataset_filepath)
reader = unicodecsv.reader(f, encoding='utf-8')

identifiers = reader.next()
contents_idx = identifiers.index("contents")
title_idx = identifiers.index("title")

contents, titles = zip(*[(row[contents_idx], row[title_idx]) for row in reader])

texts = preprocess(contents, args.stemmer, is_english_word)

dictionary = corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=1, no_above=0.8)

my_corpus = [dictionary.doc2bow(text) for text in texts]

In [6]:
try:
  model = load_model(model_filename, args.model)
  if args.override:
    raise IOError("Override flag set")
except IOError:
  print "Generating model ..."
  model = generate_model(args.model, my_corpus, dictionary, args.num_topics, args.num_iter)
  model.save(model_filename)

In [31]:
show_topics(args.model, model, args.num_topics, args.num_top_words, titles, my_corpus)

Topic #0 : service mi time power mobile bank phone business
Topic #1 : ad thing flight apple destination smart device book
Topic #2 : rice cooker mi snapdragon usage variant heating brand
Topic #3 : plan month edge free call speed subscription maxi
Topic #4 : broadband plan payment quota apple customer sport watch
Topic #5 : camera galaxy apple storage edge ram version priced
Topic #6 : camera front galaxy processor android ram device flagship
Topic #7 : front spec speaker ram back camera storage processor
Topic #8 : plan month data maxi mobile postpaid network call
Topic #9 : service text battery delivery android encryption apple feature

0 :  This could be the best postpaid plan in town — RM80/month for unlimited everything (top topic: 4)
1 :  iPhone SE and iPad Pro 9.7 official Malaysian pricing revealed (top topic: 5)
2 :  Your WhatsApp conversations are now encrypted end to end (top topic: 9)
3 :  TM UniFi Advance subscription now starts from RM199/month (top topic: 3)
4 :  11stre

In [32]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, my_corpus, dictionary)



PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      19.512254        1       1  0.158265  0.106817
8      15.977803        1       2 -0.221112  0.111180
5      14.683048        1       3  0.105191  0.104562
4      10.437065        1       4 -0.047707 -0.073616
0       9.894987        1       5 -0.002840 -0.044538
3       7.325275        1       6 -0.186754  0.049792
9       6.830234        1       7  0.016885 -0.086965
7       6.536736        1       8  0.126158  0.052742
2       4.998317        1       9  0.047572 -0.043745
1       3.804281        1      10  0.004343 -0.176229, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
1897  Default  164.000000          plan  164.000000  30.0000  30.0000
1546  Default  148.000000         month  148.000000  29.0000  29.0000
2601  Default   62.000000          data   62.000000  28.0000  28.0000
357   Default   59.000000          maxi   59.000000  27.0000  27.0000
2525  Default   69.000000         front   69.000000  26.0000  26.0000
2106  Default   94.000000          edge   94.000000  25.0000  25.0000
2785  Default   46.000000      postpaid   46.000000  24.0000  24.0000
260   Default   40.000000          free   40.000000  23.0000  23.0000
107   Default   59.000000          call   59.000000  22.0000  22.0000
1011  Default  101.000000        camera  101.000000  21.0000  21.0000
391   Default   18.000000          rice   18.000000  20.0000  20.0000
338   Default   67.000000           ram   67.000000  19.0000  19.0000
1140  Default   43.000000      offering   43.000000  18.0000  18.0000
134   Default   43.000000            mi   43.000000  17.0000  17.0000
1903  Default   65.000000        mobile   65.000000  16.0000  16.0000
2399  Default   51.000000         apple   51.000000  15.0000  15.0000
2179  Default   35.000000     broadband   35.000000  14.0000  14.0000
1350  Default   59.000000         speed   59.000000  13.0000  13.0000
1437  Default   56.000000     processor   56.000000  12.0000  12.0000
301   Default   47.000000       service   47.000000  11.0000  11.0000
20    Default   47.000000          spec   47.000000  10.0000  10.0000
247   Default   38.000000       network   38.000000   9.0000   9.0000
2882  Default   65.000000       storage   65.000000   8.0000   8.0000
941   Default   32.000000    snapdragon   32.000000   7.0000   7.0000
2856  Default   40.000000          inch   40.000000   6.0000   6.0000
2436  Default   22.000000  subscription   22.000000   5.0000   5.0000
2898  Default   52.000000       battery   52.000000   4.0000   4.0000
691   Default   48.000000         offer   48.000000   3.0000   3.0000
2141  Default   33.000000     unlimited   33.000000   2.0000   2.0000
2801  Default   16.000000            ad   16.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
573   Topic10    1.389622        adding    2.845059   2.5525  -6.0455
332   Topic10    1.389625       feeling    2.903861   2.5320  -6.0455
360   Topic10    1.389622        object    2.903860   2.5320  -6.0455
644   Topic10    1.389621          plug    2.903860   2.5320  -6.0455
1880  Topic10    2.713073          trip    5.952709   2.4833  -5.3764
272   Topic10    1.389623       created    3.005128   2.4978  -6.0455
244   Topic10    1.389622      solution    3.005128   2.4978  -6.0455
148   Topic10    3.374801         drive    8.107867   2.3925  -5.1582
490   Topic10    2.713081      interest    6.991441   2.3224  -5.3764
2746  Topic10    3.345883      computer    9.678374   2.2069  -5.1668
2053  Topic10    3.985585         smart   14.519979   1.9762  -4.9918
153   Topic10    6.703985         thing   35.068933   1.6144  -4.4718
2440  Topic10    2.713074          ride    9.346327   2.0321  -5.3764
919   Topic10    2.646077        making    9.614485   1.9789  -5.4014
1662  Topic10    3.

In [7]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, my_corpus, dictionary)

/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormatter._type_printers_default is deprecated: use @default decorator instead.
  def _type_printers_default(self):
/usr/local/lib/python2.7/site-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_for

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      30.199035        1       1  0.104082  0.184982
4      15.571528        1       2 -0.223597  0.016330
1      13.488578        1       3 -0.220154  0.050669
9      10.510970        1       4  0.003450 -0.066944
2       8.540344        1       5  0.084784  0.085647
5       5.684242        1       6  0.051086 -0.094108
0       5.374674        1       7  0.031918 -0.093433
3       5.067513        1       8  0.069257  0.009196
6       3.434300        1       9  0.043047 -0.065604
8       2.128816        1      10  0.056126 -0.026736, topic_info=     Category        Freq          Term       Total  loglift  logprob
term                                                                 
1897  Default  168.000000          plan  168.000000  30.0000  30.0000
1546  Default  151.000000         month  151.000000  29.0000  29.0000
2106  Default   92.000000          edge   92.000000  28.0000  28.0000
134   Default   42.000000            mi   42.000000  27.0000  27.0000
2601  Default   60.000000          data   60.000000  26.0000  26.0000
1350  Default   62.000000         speed   62.000000  25.0000  25.0000
2399  Default   48.000000         apple   48.000000  24.0000  24.0000
1011  Default  100.000000        camera  100.000000  23.0000  23.0000
2525  Default   71.000000         front   71.000000  22.0000  22.0000
2785  Default   46.000000      postpaid   46.000000  21.0000  21.0000
357   Default   60.000000          maxi   60.000000  20.0000  20.0000
2179  Default   37.000000     broadband   37.000000  19.0000  19.0000
2500  Default   96.000000        galaxy   96.000000  18.0000  18.0000
1140  Default   44.000000      offering   44.000000  17.0000  17.0000
391   Default   18.000000          rice   18.000000  16.0000  16.0000
2858  Default   48.000000       android   48.000000  15.0000  15.0000
2141  Default   31.000000     unlimited   31.000000  14.0000  14.0000
2856  Default   40.000000          inch   40.000000  13.0000  13.0000
338   Default   72.000000           ram   72.000000  12.0000  12.0000
1903  Default   65.000000        mobile   65.000000  11.0000  11.0000
301   Default   47.000000       service   47.000000  10.0000  10.0000
107   Default   59.000000          call   59.000000   9.0000   9.0000
1437  Default   58.000000     processor   58.000000   8.0000   8.0000
1415  Default   25.000000         power   25.000000   7.0000   7.0000
20    Default   51.000000          spec   51.000000   6.0000   6.0000
750   Default   19.000000        update   19.000000   5.0000   5.0000
2882  Default   69.000000       storage   69.000000   4.0000   4.0000
2801  Default   16.000000            ad   16.000000   3.0000   3.0000
1337  Default   34.000000         model   34.000000   2.0000   2.0000
2853  Default   20.000000       speaker   20.000000   1.0000   1.0000
...       ...         ...           ...         ...      ...      ...
2878  Topic10    1.918211         audio   11.019609   2.1013  -5.1426
598   Topic10    1.101233        desire    4.275827   2.4931  -5.6975
1558  Topic10    1.625635         sound    8.886320   2.1510  -5.3080
2820  Topic10    0.995193      finished    3.661213   2.5470  -5.7988
2853  Topic10    2.181602       speaker   20.140929   1.6269  -5.0139
1337  Topic10    2.450793         model   34.183414   1.2143  -4.8975
2230  Topic10    1.101232       forward    5.946100   2.1633  -5.6975
3057  Topic10    1.101235          boom    6.274767   2.1095  -5.6975
1880  Topic10    1.059463          trip    5.805944   2.1485  -5.7362
2381  Topic10    1.101236       packing    6.416635   2.0871  -5.6975
2856  Topic10    2.150028          inch   40.359044   0.9173  -5.0285
2141  Topic10    1.925019     unlimited   31.732165   1.0472  -5.1390
177   Topic10    0.974773     confirmed    5.271338   2.1618  -5.8195
2668  Topic10    1.018695        stereo    5.992828   2.0776  -5.7754
2706  Topic10    1.